# Testing

[Link to the website](https://www.thomasnet.com/)

In [40]:
import requests
from bs4 import BeautifulSoup
from traceback import print_exc

from datetime import datetime
import time

import numpy as np 
import pandas as pd
import math

In [21]:
headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36"
}

payload = {
    # "WTZO": "Find Suppliers",
    "cov": "NA",
    "heading": 21650809,
    "searchsource": "suppliers",
    "searchterm": "hydraulic cylinders",
    # "searchx": "true",
    "what": "hydraulic cylinders",
    # "which": "prod",
    "pg": 1
}

BASE_URL = "https://www.thomasnet.com/nsearch.html"

In [22]:
page = requests.get(BASE_URL, params=payload)
soup = BeautifulSoup(page.text,"lxml")

total_suppliers = soup.find('p',class_="supplier-search-results__subheader").findAll('b')[-1].text
print(f"{total_suppliers} suppliers found")

n_suppliers = len(soup.findAll('div',class_="supplier-search-results__card"))
print(f"{n_suppliers} found on this page")

number_of_pages = math.ceil(float(total_suppliers)/float(n_suppliers))
print(f"Total Pages: {number_of_pages}")

772 suppliers found
25 found on this page
Total Pages: 31


In [24]:
def generate_payloads(keyword: str, page_num: int):
    payloads = []
    for i in range(page_num):
        payload = {
            "cov": "NA",
            "heading": 21650809,
            "searchsource": "suppliers",
            "searchterm": keyword,
            "what": keyword,
            "pg": i+1
        }
        payloads.append(payload)

    return payloads

x = generate_payloads('hydraulic cylinders',number_of_pages)

In [52]:
collected_data = []

keyword = 'hydraulic cylinders'
page = requests.get(BASE_URL, params=x[0])
soup = BeautifulSoup(page.text,"lxml")

suppliers = soup.findAll('div',class_="supplier-search-results__card")
for sup in suppliers:
    card_data = {
        "company_id":"",
        "company_name":"",
        "company_type":"",
        "annual_revenue":"",
        "year_founded":"",
        "num_employees":"",
        "location":"",
        "company_url":"",
        "brands":"",
        "description":"",
        "url":"",
        "telephone":"",
        "searchterm":keyword
    }
    try:
        header = sup.find('header',class_='profile-card__header')
        card_data['company_id'] = eval(sup.get('data-impression-tracking'))['company_id']
        card_data['company_name'] = header.find('h2',class_='profile-card__title').text.strip()
        card_data['url'] = 'https://www.thomasnet.com'+header.find('h2',class_='profile-card__title').find('a').get('href')
        card_data['telephone'] = sup.find('a',{'data-conversion_action':'Call'}).get('href')

        sup_data = sup.find('div',class_='profile-card__supplier-data')
        card_data['location'] = sup_data.find('span',class_='profile-card__location').text.replace('ico-map','').strip()
        card_data['company_type'] = sup_data.find('span',{'data-content':'Company Type'}).text.strip()
        try:
            card_data['annual_revenue'] = sup_data.find('span',{'data-content':'Annual Revenue'}).text.strip()
        except:
            pass
        try:
            card_data['num_employees'] = sup_data.find('span',{'data-content':'Number of Employees'}).text.strip()
        except:
            pass
        try:
            card_data['year_founded'] = sup_data.find('span',{'data-content':'Year Founded'}).text.strip()
        except:
            pass

        content = sup.find('div',class_='profile-card__content')
        card_data['description'] = content.findAll('p')[0].text.strip()
        card_data['company_url'] = content.find('p',{'class':'profile-card_web-link-wrap'}).find('a').get('href')
        try:
            card_data['brands'] = content.find('p',{'class':'profile-card__brands__body'}).text.strip()
        except:
            pass
    except Exception as e:
        print("Error encountered while extraction of data\n",print_exc())
        pass

    collected_data.append(card_data)

pd.DataFrame(collected_data)

,company_id,company_name,company_type,annual_revenue,year_founded,num_employees,location,company_url,brands,description,url,telephone,searchterm
0,1315677,"Worldwide Chuck Services, Inc.","Distributor*, Service Company",$1 - 4.9 Mil,1987,1-9,"Aurora, OH",http://www.worldwidechuck.com,,Distributor of air & hydraulic cylinders inclu...,https://www.thomasnet.com/profile/01315677/wor...,tel://330-562-4509,hydraulic cylinders
1,10055126,RAM Industries Inc.,"Custom Manufacturer*, Manufacturer, Service Co...",$10 - 24.9 Mil,1974,50-99,"Yorkton, SK",https://www.ramindustries.com/products-capabil...,,Manufacturer of standard and custom hydraulic ...,https://www.thomasnet.com/profile/10055126/ram...,tel://866-982-9727,hydraulic cylinders
2,1010077,"Fluid Tech Hydraulics, Inc.","Custom Manufacturer*, Distributor, Manufacturer",$1 - 4.9 Mil,1989,10-49,"Sacramento, CA",http://www.fluidtechhydraulics.com/manufacturi...,,Manufacturer and distributor of standard and c...,https://www.thomasnet.com/profile/01010077/flu...,tel://877-882-8183,hydraulic cylinders
3,30082354,"Aries Engineering Co., Inc","Manufacturer*, Custom Manufacturer",$5 - 9.9 Mil,1994,10-49,"Dundee, MI",https://www.hypercyl.com/explore-products/,,"Manufacturer of standard and custom inline, sa...",https://www.thomasnet.com/profile/30082354/ari...,tel://877-380-9579,hydraulic cylinders
4,1150392,BDI,Distributor*,$250 Mil. and over,1935,1000+,Locations,https://www.bdiexpress.com/us/en/Fluid-Power/c...,,Distributor of hydraulic & pneumatic cylinders...,https://www.thomasnet.com/branchloc.html?cid=1...,tel://866-778-5680,hydraulic cylinders
5,30851480,Innovative Hydraulics LLC,Custom Manufacturer*,$1 - 4.9 Mil,2016,10-49,"Wausau, WI",https://innovativehyd.com/,,Custom manufacturer of hydraulic cylinders for...,https://www.thomasnet.com/profile/30851480/inn...,tel://855-373-8499,hydraulic cylinders
6,129726,HYDAC Technology Corporation,Manufacturer*,,1963,1000+,Locations,https://www.hydac-na.com/,HYDAC,Manufacturer of custom hydraulic cylinders. Al...,https://www.thomasnet.com/branchloc.html?cid=1...,tel://610-266-0100,hydraulic cylinders
7,72276,"HYSECO, Inc.","Custom Manufacturer*, Distributor",$5 - 9.9 Mil,1972,10-49,"Houston, TX",http://www.hyseco-hsc.com/custom-cylinders.html,"Custom Hoists, Power Team, Yates",Distributor and custom manufacturer of hydraul...,https://www.thomasnet.com/profile/00072276/hys...,tel://877-834-5209,hydraulic cylinders
8,1166920,M & M Rogness Equipment Co.,Distributor*,$1 - 4.9 Mil,1947,10-49,"Roseville, MN",http://couplings.mmrogness.com/category/quincy...,"Aurora, Aurora Air Products, Greer, Kep-O-seal...",Distributor of hydraulic type cylinders includ...,https://www.thomasnet.com/profile/01166920/m-m...,tel://651-633-0550,hydraulic cylinders
9,1200093,Oil-Air Products,Distributor*,$25 - 49.9 Mil,1983,50-99,"Plymouth, MN",http://www.oilair.com/products/hydraulic-cylin...,,Distributor of hydraulic cylinders for food pr...,https://www.thomasnet.com/profile/01200093/oil...,tel://877-779-0967,hydraulic cylinders
